In [2]:
import numpy as np;
import sys;
import os;
import glob;
import random;
import wavio;
import copy;
from random import shuffle;
import librosa.display
import matplotlib.pyplot as plt

In [3]:
import soundfile as sf

In [56]:
sys.path.append("../../../Libs/")
sys.path.append("../../../src/SharedLibs/")

In [57]:
from SharedLibs import getFileList
from sys_untity import ConvertSR;

### Shared Data

In [37]:
sampling_rate = 20000;
desired_wav_len = 30225;#int(np.ceil(1.5 * sampling_rate));
clear_clip_file = "../../../datasets/CurrentUse/mixed_sounds/clear_clips/clear_clip.wav" 
one_half_sec_wav_name_fmt = "{}_{}.wav";
src_eng_help_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s1_processed_clear_clips/";
head_clip_save_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/head/";
middle_clip_save_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/middle/";
tail_clip_save_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/tail/";
clear_eng_help_wav_list = getFileList(src_eng_help_dir);
clear_clip = wavio.read(clear_clip_file).data.T[0];
len_of_clear_clip = len(clear_clip);

In [38]:
# print(os.path.basename(raw_eng_help_wav_list[0])[:-4])
# print(len(clear_clip))
# test_data = wavio.read(raw_eng_help_wav_list[0]).data.T[0];
# print(f"len of test_data:{len(test_data)}, shape:{test_data.shape}");
# res_data = np.append(test_data, clear_clip);
# print(f"len of res_data:{len(res_data)}, shape:{res_data.shape}");
# print(f"count of raw english help wav files:{len(raw_eng_help_wav_list)}");
# print(len([i for i in clear_clip]))
# for i in raw_eng_help_wav_list:
#     print(i)


## Data Visualization Codes

In [49]:
import librosa.display
import matplotlib.pyplot as plt

In [50]:
def plot_signal_and_augmented_signal(signal, augmented_signal, sr):
    fig, ax = plt.subplots(nrows=2)
    librosa.display.waveplot(signal, sr=sr, ax=ax[0])
    ax[0].set(title="Original signal")
    librosa.display.waveplot(augmented_signal, sr=sr, ax=ax[1])
    ax[1].set(title="Augmented signal")
    plt.show()

## putting speech signal in the head and extending clip to 1.5 Sec

In [42]:
def extend_at_head_main(suffix_word="head", upperlel_save_dir=None):
    counter = 0;
    for w in clear_eng_help_wav_list:
        tmp_data = wavio.read(w).data.T[0];
        len_data = len(tmp_data);
        result = None;
        file_name = one_half_sec_wav_name_fmt.format(os.path.basename(w)[:-4],suffix_word);
        save_full_name = os.path.join(upperlel_save_dir, file_name)
        # print(save_full_name)
        if len_data < desired_wav_len:
            result = np.append(tmp_data, clear_clip[:desired_wav_len-len_data]);
        elif len_data > desired_wav_len:
            result = tmp_data[:desired_wav_len];
        else:
            result = tmp_data;
        sf.write(save_full_name, result, sampling_rate);
        counter += 1;
        print(f"{counter}:writing wav file to {save_full_name}");

In [44]:
# extend_at_head_main(upperlel_save_dir=head_clip_save_dir);

## putting speech signal in the middle and extending clip to 1.5 Sec

In [33]:
def insert_middle_logic(wanted_len=None, src_wav_len=None):
    dif_len_pos = int(wanted_len - src_wav_len);
    half_dif_len = int(np.ceil(dif_len_pos/2));
    start_pos = half_dif_len;
    end_pos = start_pos + src_wav_len;
    return start_pos, end_pos

In [40]:
def extend_at_middle_main(suffix_word="middle",upperlel_save_dir=None):
    counter = 0;
    for w in clear_eng_help_wav_list:
        tmp_data = wavio.read(w).data.T[0];
        len_data = len(tmp_data);
        result = None;
        file_name = one_half_sec_wav_name_fmt.format(os.path.basename(w)[:-4],suffix_word);
        save_full_name = os.path.join(upperlel_save_dir, file_name);
        # print(save_full_name)
        if len_data < desired_wav_len:
            s_pos, e_pos = insert_middle_logic(wanted_len=desired_wav_len, src_wav_len=len_data);
            result = clear_clip[:s_pos].copy();
            result = np.append(result, tmp_data[:e_pos])
            if e_pos < desired_wav_len:
                result = np.append(result, clear_clip[:desired_wav_len-e_pos]);
        elif len_data > desired_wav_len:
            result = tmp_data[:desired_wav_len];
        else:
            result = tmp_data;
        sf.write(save_full_name, result, sampling_rate);
        counter += 1;
        print(f"{counter}:writing wav file to {save_full_name}");

In [45]:
# extend_at_middle_main(upperlel_save_dir=middle_clip_save_dir)

## putting speech signal in the tail and extending clip to 1.5 Sec

In [46]:
def extend_at_tail_main(suffix_word="tail", upperlel_save_dir=None):
    counter = 0;
    for w in clear_eng_help_wav_list:
        tmp_data = wavio.read(w).data.T[0];
        len_data = len(tmp_data);
        result = None;
        file_name = one_half_sec_wav_name_fmt.format(os.path.basename(w)[:-4],suffix_word);
        save_full_name = os.path.join(upperlel_save_dir, file_name)
        # print(save_full_name)
        if len_data < desired_wav_len:
            wanted_clear_clip_len = desired_wav_len-len_data;
            result = clear_clip[:wanted_clear_clip_len].copy();
            result = np.append(result, tmp_data);
        elif len_data > desired_wav_len:
            result = tmp_data[:desired_wav_len];
        else:
            result = tmp_data;
        sf.write(save_full_name, result, sampling_rate);
        counter += 1;
        print(f"{counter}:writing wav file to {save_full_name}");

In [48]:
# extend_at_tail_main(upperlel_save_dir=tail_clip_save_dir)

## mix with white noises

In [76]:
home_chat_noises_20K_dir = "../../CurrentUse/mixed_sounds/home_chat_noises/20K/"
help_onehalfsec_clips_head_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/head/";
help_onehalfsec_clips_middle_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/middle/";
help_onehalfsec_clips_tail_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/tail/";
mixed_help_wav_fm = "{}_hcn.wav";
mixed_help_save_dir_head = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s4_processed_with_home_chat_noises/head/";
mixed_help_save_dir_middle = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s4_processed_with_home_chat_noises/middle/";
mixed_help_save_dir_tail = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s4_processed_with_home_chat_noises/tail/";
# home_chat_noise_20K_fm = "{}_20K.wav";
home_chat_noises_list = getFileList(home_chat_noises_20K_dir)
# print(f"number of chat noises files:{len(home_chat_noises_list)}")

In [ ]:
sound_list_head = getFileList(help_onehalfsec_clips_head_dir);
sound_list_middle = getFileList(help_onehalfsec_clips_middle_dir);
sound_list_tail = getFileList(help_onehalfsec_clips_tail_dir);

In [70]:
# print(os.path.basename(home_chat_noises_list[0])[:-10])
# for n in home_chat_noises_list:
#     fname = home_chat_noise_20K_fm.format(os.path.basename(n)[:-10])
#     save_path = os.path.join(home_chat_noises_20K_dir,fname);
#     ConvertSR(n,save_path,sampling_rate);
#     print(f"convert to {save_path}");

In [72]:
# for t in range(10):
#     print(random.choice(home_chat_noises_list))

In [51]:
def add_white_noise(signal, noise_percentage_factor):
    noise = np.random.normal(0, signal.std(), signal.size)
    augmented_signal = signal + noise * noise_percentage_factor
    return augmented_signal

In [52]:
def random_gain(signal, min_factor=0.1, max_factor=0.12):
    gain_rate = random.uniform(min_factor, max_factor)
    augmented_signal = signal * gain_rate
    return augmented_signal

In [53]:
def time_stretch(signal, time_stretch_rate):
    """Time stretching implemented with librosa:
    https://librosa.org/doc/main/generated/librosa.effects.pitch_shift.html?highlight=pitch%20shift#librosa.effects.pitch_shift
    """
    return librosa.effects.time_stretch(signal, time_stretch_rate)